In [11]:
#!pip install git+https://github.com/microsoft/DeBERTa.git --no-deps
#!pip instll ujson
#!pip install seqeval
!pip install datasets --upgrade 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 57.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-hub-0.12.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.4.0
    Uninstalling datasets-2.4.0:
      Successfully uninstalled datasets-2.4.0


In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold 

import numpy as np

In [2]:
import numpy as np 
import pandas as pd 
from datasets import load_dataset

In [3]:
TOKEN = ' '
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(TOKEN)


In [4]:
data = load_dataset("datadrivenscience/movie-genre-prediction")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/54000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/36000 [00:00<?, ? examples/s]

In [5]:
df_tr = pd.DataFrame()
ids = []
genres = []
titles = []
texts = []
train = data["train"].iter(batch_size=1)
 
for batch in train:
    ids.append(batch["id"][0])
    genres.append(batch["genre"][0])
    titles.append(batch["movie_name"][0])
    texts.append(batch["synopsis"][0])
    
df_tr["id"] = ids
df_tr["title"] = titles
df_tr["genre"] = genres
df_tr["text"] = texts
df_tr.to_csv("data/movies_train.csv")

In [6]:
df_test = pd.DataFrame()
ids = []
genres = []
titles = []
texts = []
split = "test"
test = data[split].iter(batch_size=1)
 
for batch in test:
    ids.append(batch["id"][0])
    genres.append(batch["genre"][0])
    titles.append(batch["movie_name"][0])
    texts.append(batch["synopsis"][0])
    
df_test["id"] = ids
df_test["title"] = titles
df_test["genre"] = genres
df_test["text"] = texts
df_test.to_csv("data/movies_test.csv")

In [7]:
skf = StratifiedKFold(n_splits=15,   shuffle=True, random_state=0)

In [8]:
df_tr = pd.read_csv('data/movies_train.csv')


In [9]:
df_tr["fold"] = -1
for i, (train_index, test_index) in enumerate(skf.split(np.zeros((len(df_tr))), df_tr.genre)):
    df_tr.loc[test_index, 'fold'] = i
df_tr.to_csv("data/movies_train_15fold.csv")